## Pre-processing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import preprocessing routine
from preprocess_module import fraud_preprocessor

# call preprocessor 
full_df = fraud_preprocessor(i_flag=0)

## Feature Engineering

In [52]:
# create physician count column 
phys_count = full_df.groupby(['AttendingPhysician', 'Provider'])['AttendingPhysician'].count()\
           .reset_index(name='Phys_Count').groupby(['Provider'])['Phys_Count'].count().reset_index()


In [53]:
# create patient count column 
patient_count = full_df.groupby(['Provider', 'BeneID'])['BeneID'].count().reset_index(name='Patient_Count')\
                .groupby('Provider')['Patient_Count'].count().reset_index()

train_features1 = pd.merge(phys_count, patient_count, on='Provider')

In [54]:
# create claim count column 
claim_count = full_df.groupby(['Provider', 'ClaimID'])['ClaimID'].count().reset_index(name='Claim_Count').\
         groupby('Provider')['Claim_Count'].count().reset_index()

train_features2 = pd.merge(train_features1, claim_count, on='Provider')

In [55]:
##create service type column
prov_type = full_df.groupby(['Provider', 'PatientType'])['BeneID'].count().reset_index(name='').drop('', axis=1)
prov_type.head()

# create a dictionary provider by service type
lst_prov_type = list(zip(prov_type['Provider'], prov_type['PatientType']))

# feed in empty dict with values inpatient, outpatient, or both
prov_type_dict = {}

for i in lst_prov_type:
    if i[0] not in prov_type_dict:
        prov_type_dict[i[0]]= i[1]    
    else:
        prov_type_dict[i[0]] = 'Both'
        

# creaete new column type of service by povider
service_type = pd.DataFrame(prov_type_dict.keys(), prov_type_dict.values()).reset_index().\
               rename(columns={'index':'Service_Type', 0:'Provider'})

# add Service column 
train_features3 = pd.merge(train_features2, service_type, on='Provider')

In [61]:
#import train label df
label_train_df = pd.read_csv('Train-1542865627584.csv')

#add label column
features = pd.merge(train_features3, label_train_df, on='Provider')
features.head()

,Provider,Phys_Count,Patient_Count,Claim_Count,Service_Type,PotentialFraud
0,PRV51001,14,24,25,Both,No
1,PRV51003,44,117,132,Both,Yes
2,PRV51004,38,138,149,Outpatient,No
3,PRV51005,7,495,1165,Outpatient,Yes
4,PRV51007,10,58,72,Both,No
